In [2]:
!pip install hnswlib


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-linux_x86_64.whl size=2389210 sha256=4d9f50f5028f7b188854ce9b8b79627479562e2df975ffa5c5449a13aac00b54
  Stored in directory: /root/.cache/pip/wheels/ea/4e/27/39aebca9958719776e36fada290845a7ef10f053ad70e22ceb
Successfully built hnswlib


In [ ]:

!ls /kaggle/input/nlp-resources/test_data.parquet


/kaggle/input/nlp-resources/test_data.parquet


In [15]:
import pandas as pd


all_data=pd.read_parquet("/kaggle/input/nlp-resources/cleaned_data.parquet")
labels = pd.read_parquet("/kaggle/input/labeled/df_supervised.parquet")

In [19]:

lab = labels['cluster_label'].tolist()

text             Caption: Tasmanian berry grower Nic Hansen sho...
cluster_id                                                       4
cluster_label                                 Finance & Investment
Name: 0, dtype: object


Let's load a sentence transformer to compute the embeddings and a cross model encoder for reranking

In [20]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util

semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
xenc_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2') #they work better but they need the compute the embedding every time we have a new query

Let's now compute the context embedding we will use for answering

In [21]:
import torch
contexts = all_data['context'].tolist()
corpus_embeddings = semb_model.encode(contexts, convert_to_tensor=True, show_progress_bar=True)
#corpus_embeddings = torch.load('//kaggle/input/corpusembeddings/corpus_embeddings.pt')


Batches:   0%|          | 0/342 [00:00<?, ?it/s]

Let's now use some random sentence, one for each cluster previously identified: we will use cosine similarity to see what is the most similar context and if it belongs to the same category

In [22]:
queries = [
    "She uploaded the latest app update to the cloud-based platform without a hitch.",           # Digital/Online Services
    "The director revealed the teaser trailer at the midnight screening.",                      # Film & Television
    "The assembly line hummed as robotic arms welded the steel frame with precision.",           # Industrial & Machinery
    "He laced up his cleats just before kick-off under the stadium lights.",                    # Sports & Athletics
    "She diversified her portfolio by investing in emerging market bonds last quarter.",        # Finance & Investment
    "They sold out tickets to the championship gaming tournament in under an hour.",             # Gaming & Events
    "Guests strolled through the resort’s marble lobby, overlooking the sun-drenched terrace.",  # Hospitality & Real Estate
    "He spent the afternoon lost in a novel about existentialism and the human condition.",      # Literature & Philosophy
    "The clinic introduced a cutting-edge diagnostic tool to improve patient outcomes.",         # Healthcare & Medicine
    "She curated a capsule wardrobe to streamline her daily style routine.",                     # Lifestyle & Personal Interests
    "Lawmakers convened to draft the new regulations on data privacy and trade compliance.",    # Legal & Government
    "He savored the flaky croissant while reading the recipe for boulangerie perfection.",      # Food & Culinary
    "The candidate outlined her vision for education reform during the televised debate.",       # Politics & Public Affairs
    "Volunteers conducted a beach cleanup to protect the fragile coastal ecosystem.",           # Environment & Ecology
    "She blended acrylics to capture the vibrant hues of the urban skyline at dusk.",            # Art & Design
    "Researchers presented their groundbreaking findings on renewable energy sources.",         # Education & Research
    "The DJ mixed vintage vinyl records to curate the perfect summer soundtrack.",              # Music & Audio
    "He practiced mindfulness meditation to maintain balance amid his hectic schedule."         # Wellness & Personal Care
]


Let's compute the embeddings of these sentences, too.

In [23]:
queries_embeddings = semb_model.encode(queries, convert_to_tensor=True,show_progress_bar=True )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Let's find the most similar contexts with cosine similarity and see what category they belong to and if they are actually relevant.

We can also use the built-in function for semantic search

In [24]:
query_embeddings =  semb_model.encode(queries, convert_to_tensor=True)

util.semantic_search(query_embeddings, corpus_embeddings, score_function=util.cos_sim)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'corpus_id': 603, 'score': 0.40528178215026855},
  {'corpus_id': 642, 'score': 0.3988167345523834},
  {'corpus_id': 9173, 'score': 0.39425790309906006},
  {'corpus_id': 6954, 'score': 0.39246395230293274},
  {'corpus_id': 9056, 'score': 0.3865756690502167},
  {'corpus_id': 273, 'score': 0.380735844373703},
  {'corpus_id': 113, 'score': 0.36934608221054077},
  {'corpus_id': 6911, 'score': 0.3677324950695038},
  {'corpus_id': 851, 'score': 0.3670920133590698},
  {'corpus_id': 5821, 'score': 0.3656845688819885}],
 [{'corpus_id': 1497, 'score': 0.4757058322429657},
  {'corpus_id': 317, 'score': 0.46287935972213745},
  {'corpus_id': 6703, 'score': 0.42867356538772583},
  {'corpus_id': 8269, 'score': 0.4266166090965271},
  {'corpus_id': 10420, 'score': 0.4123404920101166},
  {'corpus_id': 1819, 'score': 0.40546783804893494},
  {'corpus_id': 2713, 'score': 0.39723342657089233},
  {'corpus_id': 10865, 'score': 0.3966255187988281},
  {'corpus_id': 10168, 'score': 0.3938640356063843},
  {'cor

We can also speed up this process by pre-normalising the embeddings and compute directly the dot-product instead of cosine similarity.

In [25]:
normalised_docs_embeddings = util.normalize_embeddings(semb_model.encode(contexts, convert_to_tensor=True))
normalised_query_embeddings = util.normalize_embeddings(semb_model.encode(queries, convert_to_tensor=True))
#norm_docs_embeddings = torch.load('/kaggle/input/normdocsembedding/norm_docs_embeddings.pt')
#norm_query_embeddings = torch.load('/kaggle/input/normqueryemb/norm_query_embeddings.pt')

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
hits = util.semantic_search(normalised_query_embeddings, normalised_docs_embeddings, score_function=util.dot_score)
hits

[[{'corpus_id': 603, 'score': 0.40528178215026855},
  {'corpus_id': 642, 'score': 0.3988167345523834},
  {'corpus_id': 9173, 'score': 0.39425790309906006},
  {'corpus_id': 6954, 'score': 0.39246395230293274},
  {'corpus_id': 9056, 'score': 0.3865756690502167},
  {'corpus_id': 273, 'score': 0.380735844373703},
  {'corpus_id': 113, 'score': 0.36934608221054077},
  {'corpus_id': 6911, 'score': 0.3677324950695038},
  {'corpus_id': 851, 'score': 0.3670920133590698},
  {'corpus_id': 5821, 'score': 0.3656845688819885}],
 [{'corpus_id': 1497, 'score': 0.4757058322429657},
  {'corpus_id': 317, 'score': 0.46287935972213745},
  {'corpus_id': 6703, 'score': 0.42867356538772583},
  {'corpus_id': 8269, 'score': 0.4266166090965271},
  {'corpus_id': 10420, 'score': 0.4123404920101166},
  {'corpus_id': 1819, 'score': 0.40546783804893494},
  {'corpus_id': 2713, 'score': 0.39723342657089233},
  {'corpus_id': 10865, 'score': 0.3966255187988281},
  {'corpus_id': 10168, 'score': 0.3938640356063843},
  {'cor

Since it took a lot of time, let's save the embedding we previously computed

In [27]:
torch.save(corpus_embeddings, 'corpus_embeddings.pt')
torch.save(normalised_docs_embeddings, 'norm_docs_embeddings.pt')
torch.save(normalised_query_embeddings, 'norm_query_embeddings.pt')

In order to load them the next time i have to do this

In [ ]:
'''corpus_embeddings = torch.load('corpus_embeddings.pt')
norm_docs_embeddings = torch.load('normalised_docs_embeddings.pt')
norm_query_embeddings = torch.load('normalised_query_embeddings.pt')'''

Let's now try and do the same thing with the cross encoder. Basically this model will be used to compute the similarity instead of using cosine similarity.

In [ ]:
import numpy as np

import textwrap  

MAX_CHARS = 150
TOP_K = 5

for query in queries:
    
    xenc_model_inputs = [[query, context] for context in contexts]
    # compute similaruty
    scores = xenc_model.predict(xenc_model_inputs)

    # i take highest TOP_K 
    top_idxs = np.argsort(-scores)[:TOP_K]

    print(f"Query: \"{query}\"")
    print("Top 5 most similar contexts:")
    print("---------------------------------------")
    for idx in top_idxs:
        full_ctx = contexts[idx]
        # hortening contex
        short_ctx = full_ctx[:MAX_CHARS] + ("..." if len(full_ctx) > MAX_CHARS else "")
       

        print(f"Score: {scores[idx]:.4f} – \"{short_ctx}\", {lab[idx]}")
    print("\n")

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

Query: "She uploaded the latest app update to the cloud-based platform without a hitch."
Top 5 most similar contexts:
---------------------------------------
Score: -5.1386 – "5. eHarmony
eHarmony is known as a platform that is top locating a soulmate and building effective relationships. In line with the supplied informatio...", Digital/Online Services
Score: -5.3458 – "The latest Google real time auto-translation app update has promised to recognise a spoken language and make a real-time transcript of the words, as w...", Digital/Online Services
Score: -5.4027 – "Read the Original Article at
Google on Wednesday outlined its vision for the Internet, which will include a broad scope of technical ambitions using t...", Digital/Online Services
Score: -5.4532 – "Business New platform transforms Excel spreadsheets into working web apps in minutes, automatically Image Credit: Craig Chew-Moulding November 13, 201...", Digital/Online Services
Score: -5.8853 – "FatCloud Announces New Free Comm

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

Query: "The director revealed the teaser trailer at the midnight screening."
Top 5 most similar contexts:
---------------------------------------
Score: -2.7779 – "'The Dark Knight Rises' Countdown: Fake Trailers and Feline Fatales
Chris Clow is a comic book expert and contributor to Batman-On-Film.com. You can f...", Film & Television
Score: -4.3955 – "210 – What Happens Nexton May 28th, 2009
IN-DEPTH REVIEW OF THE SUPERFOGEYS OVER AT “DIGITAL STRIPS!”
Wow. The Midnight Cartooner, a contributor at Di...", Film & Television
Score: -4.6109 – "Calm Down, A ‘Rogue One’ Trailer is Coming
There was plenty of buzz surrounding the upcoming addition to the Star Wars saga at San Diego Comic Con thi...", Film & Television
Score: -5.2504 – "Some Guy Who Kills People is set to be the sleeper hit of the Toronto After Dark Film Festival. Though it screened at 1:30 pm on a Sunday afternoon, i...", Film & Television
Score: -5.2556 – "Hi Folks, Have you watched this movie- 2012? Here's a little bit bac

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

...

Query: "Researchers presented their groundbreaking findings on renewable energy sources."
Top 5 most similar contexts:
---------------------------------------
Score: 1.2387 – "AUSTIN—Today, the Texas Public Policy Foundation released the research paper, The Economic Fall and Political Rise of Renewable Energy.
The paper’s au...", Finance & Investment
Score: -1.2008 – "Please enable Javascript in your browser.
Researchers worldwide are working to develop new technologies for producing clean energy. A team of research...", Industrial & Machinery
Score: -2.6774 – "Renewable Energy and Health: Me, Myself and I
In the continuing debate surrounding how, why, when, where and whether we should move decisively ahead w...", Wellness & Personal Care
Score: -2.7935 – "Coal, oil, and natural gas are still dominant sources of energy despite the fact that renewable energy is getting more and more supporters. Wind power...", Finance & Investment
Score: -3.2396 – "State Grid China teams up with German 

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

Query: "The DJ mixed vintage vinyl records to curate the perfect summer soundtrack."
Top 5 most similar contexts:
---------------------------------------
Score: -2.9979 – "Spotify released its official “Song of the Summer” list this week, and and it’s filled with tracks you’ve probably had stuck in your head over the pas...", Music & Audio
Score: -3.3983 – "Some trunk rattling awesomeness for your Mother’s Day weekend.
Ty Dolla $ign – Beach House 3 Deluxe
Last year, Ty Dolla $ign blessed the masses with t...", Music & Audio
Score: -3.6043 – "Artist(s): G.O.O.D. Music
Cruel Summer is looking more like just another summer gone by.
Review Score: 3.8
It’s been a long and grueling summer for mu...", Music & Audio
Score: -4.3465 – "
Mar
25
Sexy Funky Electro House Set Chris Humphrey
Posted (
admin
) in
House Music News
on March-25-2006
Coming from Ibiza Spain Chris Humphreys know...", Music & Audio
Score: -5.5894 – "The soundtrack to the much-hyped film House of Wax is on the shelves, thanks

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

Query: "He practiced mindfulness meditation to maintain balance amid his hectic schedule."
Top 5 most similar contexts:
---------------------------------------
Score: -3.8127 – "One of the more recent discoveries of neuroscience is that when you stare into space, gather wool, daydream, or otherwise let your mind wander, the “d...", Lifestyle & Personal Interests
Score: -4.4015 – ""My husband knew that I found Therapeutic Touch helpful and comforting but didn't believe it really worked until Tama did a session for me and nurses ...", Healthcare & Medicine
Score: -4.5501 – "All About Yoga And Why It’s Good For You
All About Yoga and Why It’s Good For You
Yoga’s popularity is on the rise. No matter where you live, it is li...", Wellness & Personal Care
Score: -4.7199 – "QV, Sewickley Academy coaches value preseason practice for winter sports
TribLIVE Sports Videos
Yoga is part of one team's workouts as squads at Quake...", Sports & Athletics
Score: -5.1472 – "It quotes self-help author Ga

Very nice: as we can see in many cases the labels match!

When we use an embedding model with cosine similarity, we can pre-compute the embeddings in our data set and index them to speed-up the search. There are techniques for Approximate Nearest Neighbor (ANN), which use clustering to index the embedding space and speed-up the search process. Let's do indexing with HNSWLIB

In [35]:
import hnswlib

index = hnswlib.Index(space='cosine', dim=corpus_embeddings.size(1))

Now we can index our data. The index we compute can be saved and loaded, so we can check if it is already availabel and load it (this will save time)

In [36]:
import os

# Define hnswlib index path
index_path = "./hnswlib.index"

# Load index if available
if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
# Else index data collection
else:
    # Initialise the index
    print("Start creating HNSWLIB index")
    index.init_index(max_elements=corpus_embeddings.size(0), ef_construction=400, M=64)
    #  Compute the HNSWLIB index (it may take a while)
    index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
    # Save the index to a file for future loading
    print("Saving index to:", index_path)
    index.save_index(index_path)

Loading index...


Let's see if it's actually faster by measuring the running time

In [40]:
from datetime import datetime

# Search using index
t_start = datetime.now()
_ = index.knn_query(queries_embeddings[0].cpu(), k=128)
t_stop = datetime.now()
print(f"Search time with index: {t_stop - t_start}")

# Search without index
t_start = datetime.now()
_ = util.semantic_search(queries_embeddings, corpus_embeddings, score_function=util.cos_sim, top_k=128)
t_stop = datetime.now()
print(f"Search time without index: {t_stop - t_start}")

Search time with index: 0:00:00.002146
Search time without index: 0:00:00.003131


Reranking
Since the cross encoder gives better results but is slower than cosine similarity, we can take advantage of both: We can do a first search with bi-encoder models and then re-rank the top- k  results with a cross-encoder. We call this approach retrieve and re-rank.

Let's define a new random query (we could have used one of those generated before)

In [41]:
query = "Who is the president of the United States?"
query_embedding = semb_model.encode(query, convert_to_tensor=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Search using ANN index

In [42]:
corpus_ids, distances = index.knn_query(query_embedding.cpu(), k=3)
scores = 1 - distances

print("Cosine similarity model search results")
print(f"Query: \"{query}\"")
print("---------------------------------------")
for idx, score in zip(corpus_ids[0], scores[0]):
    print(f"Score: {score:.4f}\nDocument: \"{contexts[idx]}\"\n\n")

Cosine similarity model search results
Query: "Who is the president of the United States?"
---------------------------------------
Score: 0.4851
Document: "UUA Presidential Search Committee begins its work
Created by 2010 General Assembly vote, committee will pick two nominees for 2017 UUA presidential election.
That system is changing. In 2010, the General Assembly approved the creation of a Presidential Search Committee and charged it with selecting at least two nominees for president. The General Assembly also shortened the president's term of office from two four-year terms to a single six-year term.
Members of the search committee were selected in 2013. This spring they are exploring how a system might be created that would be open to more people. "The goal of the committee is to not just select candidates," said the Rev. Dr. Matthew Johnson, co-chair, "but to think creatively and systematically about what the job entails and to broaden the circle of people willing to consider thi

Let's add the re-ranking

In [43]:
corpus_ids, _ = index.knn_query(query_embedding.cpu(), k=128)

model_inputs = [(query, contexts[idx]) for idx in corpus_ids[0]]
cross_scores = xenc_model.predict(model_inputs)

print("Cross-encoder model re-ranking results")
print(f"Query: \"{query}\"")
print("---------------------------------------")
for idx in np.argsort(-cross_scores)[:3]:
    print(f"Score: {cross_scores[idx]:.4f}\nDocument: \"{contexts[corpus_ids[0][idx]]}\"\n\n")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Cross-encoder model re-ranking results
Query: "Who is the president of the United States?"
---------------------------------------
Score: 0.7632
Document: "Why It Makes A Difference!
The President of the United States is a liar. Not a tiny weenie white liar, but a Sociopathic Liar. His word is not worth the air he is allowed to use to say it. Why Does It Make A Difference? It is because the leader of the Free World MUST be Trusted. When the President of the United States is willing to repeatedly Testify to lies, then who should our Children look up to? Sports Figures? When everyone within the Obama Administration is willing to SAY and do anything regardless of TRUTH, then why would anyone in the WORLD believe in America or Her Freedoms.
The recent admission to IRS intimidation against “Tea Party” shows the level of tyrannical Leadership that Mr. Obama is willing to stoop. With the passage of the Obama Care TAX legislation, imagine the CONTROL that our Massive “Progressive” Government w

Finally, let's see if our dataset is big enough to give correct answers to generic questions just by finding the most similar question in the dataset and giving its associated answer. It should work kind of like a chatbot

Let's first compute the answers' embeddings

In [44]:
answers = all_data['answer'].tolist()
answers_embeddings = semb_model.encode(answers, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

In [45]:
questions = all_data['question'].tolist()
questions_embeddings = semb_model.encode(questions, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/342 [00:00<?, ?it/s]

In [46]:
# Create empty index
hnswlib_index = hnswlib.Index(space='cosine', dim=questions_embeddings.size(1))

# Define hnswlib index path
index_path = "./emp_dialogue_hnswlib.index"

# Load index if available
if os.path.exists(index_path):
    print("Loading index...")
    hnswlib_index.load_index(index_path)
# Else index data collection
else:
    # Initialise the index
    print("Start creating HNSWLIB index")
    hnswlib_index.init_index(max_elements=questions_embeddings.size(0), ef_construction=400, M=64)
    #  Compute the HNSWLIB index (it may take a while)
    hnswlib_index.add_items(questions_embeddings.cpu(), list(range(len(questions_embeddings))))
    # Save the index to a file for future loading
    print("Saving index to:", index_path)
    hnswlib_index.save_index(index_path)

Start creating HNSWLIB index
Saving index to: ./emp_dialogue_hnswlib.index


In [ ]:
import numpy as np
def get_response(
    message: str,
    questions_embeddings,     # array/tensor of questions embeddings
    answers: list[str],       # answers list
    index,                    # hnswlib.Index with questions_embeddings
    re_ranking_model=None,    #  cross-encoder
    top_k: int = 32
) -> str:
   
    message_emb = semb_model.encode(message, convert_to_tensor=True).cpu().numpy()

   
    corpus_ids, _ = index.knn_query(message_emb, k=top_k)

    
    candidate_idxs = corpus_ids[0]
    if re_ranking_model is not None:
        model_inputs = [
            (message, answers[i])
            for i in candidate_idxs
        ]
        cross_scores = re_ranking_model.predict(model_inputs) 
        best_pos = np.argmax(cross_scores)
        best_idx = candidate_idxs[best_pos]
    else:
       
        best_idx = candidate_idxs[0]
    return answers[best_idx]

In [ ]:
chatbot_response = get_response(
    "who is einstein?", 
    questions_embeddings,                
    answers,                            
    hnswlib_index,                        
    re_ranking_model=xenc_model,          
    top_k=32                              
)

print("Chatbot says:", chatbot_response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chatbot says: Albert Einstein's most famous paper was the Special Theory of Relativity. It revealed that the speed of light is a constant against which even time and space lose their absolute meaning.


As we could imagine, not the best result even thought if we are lucky the responses actully make some sense. Of course this is because the dataset we have is not that big